In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.io_utils import read_data, clean_data

df = read_data("data/ecommerce_sales.csv")

df_clean = clean_data(df)

print("Here is a sample of our data:")
df_clean.sample(3)

Here is a sample of our data:


,order_id,date,city,category,price,units,revenue
1060,101060,2024-01-31,Göteborg,Clothing,737.33,3,2211.99
2241,102241,2024-06-15,Göteborg,Home,640.68,1,640.68
2150,102150,2024-02-11,Stockholm,Home,765.92,3,2297.76
